In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("B2.csv", delimiter=";")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%d/%m/%Y %H:%M")

In [3]:
for i in ["TRIP", "TEMPERATURE", "FUEL", "PERSONS"]:
    df[i].fillna(method="ffill", inplace=True)

C:\Users\canut\AppData\Local\Temp\ipykernel_14088\2756122955.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[i].fillna(method="ffill", inplace=True)


In [10]:
df["ALTITUDE"].fillna(
    df["ALTITUDE"].rolling(window=3, min_periods=1).mean(), inplace=True
)

df["ALTITUDE"] = df["ALTITUDE"].round(0)

In [11]:
dn = list(df.select_dtypes(include=np.number).columns)

for i in dn:
    df[i] = df[i].astype(int)

In [12]:
df[df["ALTITUDE"].isna()].index

Index([], dtype='int64')

In [31]:
for i in [366, 367]:
    df.at[i, "ALTITUDE"] = 106

In [33]:
df.at[387, "ALTITUDE"] = 98

In [17]:
df

,TIMESTAMP,SPEED,ALTITUDE,TRIP,TEMPERATURE,FUEL,PERSONS
0,2024-04-21 16:50:00,0,33,GO,22,REPSOL,1
1,NaT,4,33,GO,22,REPSOL,1
2,NaT,9,33,GO,22,REPSOL,1
3,NaT,0,33,GO,22,REPSOL,1
4,NaT,13,33,GO,22,REPSOL,1
...,...,...,...,...,...,...,...
855,NaT,30,7,GO,22,V2,1
856,NaT,24,7,GO,22,V2,1
857,NaT,19,7,GO,22,V2,1
858,NaT,4,7,GO,22,V2,1


In [18]:
df.to_csv("fin_V2.csv")

In [3]:
df = pd.read_csv("end_REP.csv", delimiter=";")

In [13]:
# df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%Y-%m-%d %H:%M:%S")

df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%d/%m/%Y %H:%M")

In [14]:
lr = df[df["TIMESTAMP"].notnull()].index
lr

Index([  0,  69,  70, 151, 152, 238, 239, 321, 322, 402, 403, 513, 514, 611,
       612, 685, 686, 759, 760, 859],
      dtype='int64')

In [15]:
def get_even_lists(lst):
    even_pos = []
    odd_pos = []
    lim = len(lst)

    for i in range(len(lst)):
        if i % 2 == 1:
            odd_pos.append(lst[i])
        else:
            even_pos.append(lst[i])
    return odd_pos, even_pos

In [16]:
odd_l, even_l = get_even_lists(lr)
odd_l, even_l

([69, 151, 238, 321, 402, 513, 611, 685, 759, 859],
 [0, 70, 152, 239, 322, 403, 514, 612, 686, 760])

In [17]:
lg = []
for i, j in zip(even_l, odd_l):
    l_i = pd.date_range(
        start=pd.to_datetime(str(df["TIMESTAMP"][i])),
        end=pd.to_datetime(str(df["TIMESTAMP"][j])),
        periods=(j - i + 1),
    ).round(freq="min")
    print(l_i[-1])
    lg.append(l_i)

ld = [x for xs in lg for x in xs]
df["TIMESTAMP"] = ld

2024-04-21 16:57:00
2024-04-21 17:15:00
2024-04-23 18:32:00
2024-04-25 09:50:00
2024-04-25 11:08:00
2024-04-26 08:23:00
2024-04-26 13:54:00
2024-05-03 08:26:00
2024-05-03 08:46:00
2024-05-03 16:41:00


In [154]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [49]:
df.head()

,Unnamed: 0,TIMESTAMP,SPEED,ALTITUDE,TRIP,TEMPERATURE,FUEL,PERSONS
0,0,2024-04-21 16:50:00,0,33,GO,22,REPSOL,1
1,1,2024-04-21 16:50:00,4,33,GO,22,REPSOL,1
2,2,2024-04-21 16:50:00,9,33,GO,22,REPSOL,1
3,3,2024-04-21 16:50:00,0,33,GO,22,REPSOL,1
4,4,2024-04-21 16:50:00,13,33,GO,22,REPSOL,1


In [58]:
df = pd.read_csv("final_1.csv", delimiter=";")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%d/%m/%Y %H:%M")

In [81]:
df['day_formatted'] = df['TIMESTAMP'].dt.strftime("%d/%m/%Y")

df['day_fuel'] = df['day_formatted'] + "_" + df['FUEL'].astype(str)
df['day_fuel'] = df['day_fuel'] + "_" + df['COUNT_TRIPS'].astype(str)

# df.drop(['day_c'], axis=1, inplace=True)

In [82]:
l_g = list(df['FUEL'].unique())
l_g

['BP', 'REPSOL', 'V2']

In [83]:
df[df['FUEL'] == 'BP'].groupby('day_fuel')[['SPEED','ALTITUDE','PERSONS']].agg(['mean', np.median, 'max']).round(0)

C:\Users\canut\AppData\Local\Temp\ipykernel_12200\2590579173.py:1: FutureWarning: The provided callable <function median at 0x00000294D07DC720> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'median' instead.
  df[df['FUEL'] == 'BP'].groupby('day_fuel')[['SPEED','ALTITUDE','PERSONS']].agg(['mean', np.median, 'max']).round(0)


SPEED             ALTITUDE             PERSONS           
                 mean median  max     mean median  max    mean median max
day_fuel                                                                 
05/04/2024_BP_1  18.0   18.0   43     39.0   39.0   50     3.0    3.0   3
05/04/2024_BP_2  14.0   14.0   38     39.0   38.0   50     3.0    3.0   3
06/04/2024_BP_1  51.0   44.0  112     46.0   32.0  125     3.0    3.0   3
06/04/2024_BP_2  65.0   70.0  126     46.0   27.0  125     1.0    1.0   1
13/04/2024_BP_1  52.0   39.0  115     59.0   43.0  125     2.0    2.0   2

In [134]:
agg_c = {'SPEED': ['mean', 'median', 'max'], 
         'ALTITUDE': ['mean', 'median','max'],
         'PERSONS': ['max'], 
         'TEMPERATURE': 'mean'}

In [135]:
for i in l_g:
    print(df[df['FUEL'] == i].groupby('day_d').agg(agg_c).round(0))

                SPEED             ALTITUDE             PERSONS TEMPERATURE
                 mean median  max     mean median  max     max        mean
day_d                                                                     
05/04/2024_1_BP  18.0   18.0   43     39.0   39.0   50       3        21.0
05/04/2024_2_BP  14.0   14.0   38     39.0   38.0   50       3        21.0
06/04/2024_1_BP  51.0   44.0  112     46.0   32.0  125       3        22.0
06/04/2024_2_BP  65.0   70.0  126     46.0   27.0  125       1        26.0
13/04/2024_1_BP  52.0   39.0  115     59.0   43.0  125       2        19.0
                    SPEED             ALTITUDE             PERSONS TEMPERATURE
                     mean median  max     mean median  max     max        mean
day_d                                                                         
14/04/2024_1_REPSOL  22.0   19.0   67    104.0  105.0  113       4        22.0
14/04/2024_2_REPSOL  41.0   26.0  123     69.0   87.0  125       4        23.0
19/04

In [115]:
def route(fuel):
    results = {
    'total': df[df['FUEL'] == fuel].groupby('day_fuel')['ROUTE'].count(),
    'city_tot': df[(df['FUEL'] == fuel) & (df['ROUTE'] == 'CITY')].groupby('day_fuel')['ROUTE'].count(),
    'high_tot': df[(df['FUEL'] == fuel) & (df['ROUTE'] == 'HIGHSPEED')].groupby('day_fuel')['ROUTE'].count()    
    }

    results_df = pd.DataFrame(results).fillna(0)
    results_df.reset_index(inplace=True)

    results_df['city_%'] = round(results_df['city_tot']/results_df['total']*100,2)
    
    return results_df

In [116]:
route('BP')

,day_fuel,total,city_tot,high_tot,city_%
0,05/04/2024_BP_1,43,43,0.0,100.00
1,05/04/2024_BP_2,59,59,0.0,100.00
2,06/04/2024_BP_1,46,24,22.0,52.17
3,06/04/2024_BP_2,55,20,35.0,36.36
4,13/04/2024_BP_1,83,45,38.0,54.22
